In [ ]:
import requests
import pandas as pd
import glob
import os

In [20]:
# PROVINCIAL SOLO LIMA METROPOLITANA
url = "https://api.resultadoserm2022.onpe.gob.pe/results/03/140100"
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
}

resp = requests.get(url, headers=headers)

data = resp.json()['results']

fecha = resp.json()['generals']['actData']['FECHA']
hora = resp.json()['generals']['actData']['HORA']
avance = resp.json()['generals']['actData']['POR_AVANCE']

if data != []:
    df = pd.DataFrame(data)
    df['fecha'] = fecha
    df['hora'] = hora
    df['avance'] = avance
    df.to_csv(f"lima_metropolitana.csv")

In [19]:
# REGIONAL
for departamento in range(1,26):
    url = f"https://api.resultadoserm2022.onpe.gob.pe/results/01/{departamento:02d}0000"
    headers = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
    }

    resp = requests.get(url, headers=headers)

    data = resp.json()['results']

    fecha = resp.json()['generals']['actData']['FECHA']
    hora = resp.json()['generals']['actData']['HORA']
    avance = resp.json()['generals']['actData']['POR_AVANCE']

    if data != []:
        df = pd.DataFrame(data)
        df['departamento'] = f'{departamento:02d}'
        df['fecha'] = fecha
        df['hora'] = hora
        df['avance'] = avance
        df.to_csv(f"resultados_regionales/{departamento:02d}0000.csv")


# setting the path for joining multiple files
files = os.path.join("resultados_regionales", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df_resultados = pd.concat(map(pd.read_csv, files), ignore_index=True).drop(columns=['Unnamed: 0'])
df_resultados.to_csv('resultados_regional.csv')
df_resultados

Resultant CSV after joining all CSV files at a particular location...


,C_CODI_AGP,AGRUPACION,IMAGENAGRUPACION,PORC_AVANCE_ACTA,ACTAS_COMPUTADA,TOTAL_MESAS,TOTAL_VOTOS,NLISTA,POR_VALIDOS,POR_EMITIDOS,departamento,fecha,hora,avance
0,99.0,PIURA RENACE,NaN,100.000,"3,123","3,123","31,904",00000099.jpg,5.218,4.345,19,03/10/2022,08:16 h,64.3254
1,98.0,ORGANIZACION POLITICA UNIDAD REGIONAL,NaN,100.000,"3,123","3,123","156,891",00000098.jpg,25.662,21.367,19,03/10/2022,08:16 h,64.3254
2,95.0,MOVIMIENTO INDEPENDIENTE FUERZA REGIONAL,NaN,100.000,"3,123","3,123","75,203",00000095.jpg,12.301,10.242,19,03/10/2022,08:16 h,64.3254
3,97.0,REGION PARA TODOS,NaN,100.000,"3,123","3,123","102,106",00000097.jpg,16.701,13.906,19,03/10/2022,08:16 h,64.3254
4,96.0,CONTIGO REGION,NaN,100.000,"3,123","3,123","96,637",00000096.jpg,15.807,13.161,19,03/10/2022,08:16 h,64.3254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,7.0,AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL,NaN,100.000,"1,750","1,750","81,558",00000007.jpg,26.761,21.909,9,03/10/2022,08:16 h,82.0056
299,NaN,TOTAL DE VOTOS VÁLIDOS,NaN,NaN,NaN,NaN,"304,770",NaN,100.000,81.872,9,03/10/2022,08:16 h,82.0056
300,80.0,VOTOS EN BLANCO,NaN,"6,285.149","1,750","1,993","45,360",NaN,NaN,12.185,9,03/10/2022,08:16 h,82.0056
301,81.0,VOTOS NULOS,NaN,"6,285.149","1,750","1,993","22,123",NaN,NaN,5.943,9,03/10/2022,08:16 h,82.0056
